In [1]:
!pip install "sagemaker>=2.48.0" "transformers==4.6.1" "datasets[s3]==1.6.2" --upgrade

     |████████████████████████████████| 447 kB 4.7 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 62.9 MB/s eta 0:00:01
     |████████████████████████████████| 221 kB 84.9 MB/s eta 0:00:01
     |████████████████████████████████| 7.2 MB 30.4 MB/s eta 0:00:01
     |████████████████████████████████| 130 kB 80.3 MB/s eta 0:00:01
     |████████████████████████████████| 73 kB 3.8 MB/s  eta 0:00:01
     |████████████████████████████████| 52 kB 2.2 MB/s  eta 0:00:01
     |████████████████████████████████| 52 kB 177 kB/s  eta 0:00:01
     |████████████████████████████████| 51 kB 702 kB/s  eta 0:00:01
     |████████████████████████████████| 50 kB 10.3 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 343 kB/s  eta 0:00:01
     |████████████████████████████████| 48 kB 8.5 MB/s  eta 0:00:01
     |████████████████████████████████| 48 kB 9.0 MB/s  eta 0:00:01
  Created wheel for sagemaker: filename=sagemaker-2.63.2-py2.py3-none-any.whl size=622823 sha256=9c91a0c7fafb634

In [1]:
import sagemaker.huggingface
import sagemaker

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

print(f"IAM role arn used for running training: {role}")
print(f"S3 bucket used for storing artifacts: {sess.default_bucket()}")

IAM role arn used for running training: arn:aws:iam::847380964353:role/service-role/AmazonSageMaker-ExecutionRole-20200409T140000
S3 bucket used for storing artifacts: sagemaker-us-west-2-847380964353


In [22]:
import pandas as pd
import os
from datasets import load_dataset

df_article_summary_full = pd.read_parquet('./meta_description.parquet', engine='pyarrow')
df_article_summary_full[['original_text','meta_descrption']].to_csv('./total.csv',index=False)

total_data = pd.read_csv('./total.csv')
x = total_data[-total_data['meta_descrption'].isnull()]
x.columns = ['article','summarization']

# use csv file to test 
x[:1000].to_csv('./train.csv',index=False,encoding='utf-8')
x[1000:1200].to_csv('./test.csv',index=False,encoding='utf-8')
x[1200:1400].to_csv('./dev.csv',index=False,encoding='utf-8')

In [25]:
import boto3
prefix='hk01'

bucket = sess.default_bucket() 
boto3.Session().resource("s3").Bucket(bucket).Object(
    os.path.join(prefix, "train/train.csv")
).upload_file("./train.csv")
boto3.Session().resource("s3").Bucket(bucket).Object(
    os.path.join(prefix, "test/test.csv")
).upload_file("./test.csv")

In [33]:
training_input_path = f's3://{sess.default_bucket()}/{prefix}/train/train.csv'
test_input_path = f's3://{sess.default_bucket()}/{prefix}/test/test.csv'

In [40]:
git_config = {'repo': 'https://github.com/huggingface/transformers.git','branch': 'v4.6.1'} # v4.6.1 is referring to the `transformers_version` you use in the estimator.

In [41]:
hyperparameters={'per_device_train_batch_size':1,
                 'per_device_eval_batch_size': 1,
                 'model_name_or_path': 'google/mt5-base',
                 'train_file':'/opt/ml/input/data/train/train.csv',
                 'validation_file':'/opt/ml/input/data/test/test.csv',
                 'test_file':'/opt/ml/input/data/test/test.csv',
                 'do_train': True,
                 'do_predict': True,
                 'do_eval': True,
                 'text_column':'article',
                 'summary_column':'summarization',
                 'save_total_limit':3,
                 'num_train_epochs': 1,
                 'predict_with_generate': True,
                 'output_dir': '/opt/ml/model',
                 'num_train_epochs': 1,
                 'learning_rate': 5e-5,
                 'seed': 7,
                 'fp16': False,
                 'source_prefix': "summarize: ",
                 'eval_steps': 1000,
                 }

# configuration for running training on smdistributed Data Parallel
#distribution = {'smdistributed':{'dataparallel':{ 'enabled': True }}}

In [47]:
from sagemaker.huggingface import HuggingFace

# create the Estimator
huggingface_estimator = HuggingFace(
      entry_point='run_summarization.py', # script
      source_dir='./examples/pytorch/summarization', # relative path to example
      git_config=git_config,
      instance_type='ml.p3.16xlarge', # here better to use ml.p3dn.24xlarge if available
      instance_count=1,
      volume_size=500,
      transformers_version='4.6',
      pytorch_version='1.7',
      py_version='py36',
      role=role,
      base_job_name='mt5', 
      hyperparameters = hyperparameters
)


In [48]:
huggingface_estimator.fit({'train':training_input_path,'test':test_input_path})

2021-11-03 06:15:58 Starting - Starting the training job...
2021-11-03 06:16:21 Starting - Launching requested ML instancesProfilerReport-1635920152: InProgress
.........
2021-11-03 06:17:51 Starting - Preparing the instances for training......
2021-11-03 06:18:52 Downloading - Downloading input data...
2021-11-03 06:19:22 Training - Downloading the training image...............
2021-11-03 06:21:43 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-11-03 06:21:41,845 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-11-03 06:21:41,918 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-11-03 06:21:44,937 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-11-03 06:21:45,265 sagemaker-training-toolkit INFO     Installing dependencies from r

In [49]:
predictor = huggingface_estimator.deploy(1,"ml.g4dn.xlarge")

-------!

In [ ]:
conversation = '''Jeff: Can I train a 🤗 Transformers model on Amazon SageMaker? 
    Philipp: Sure you can use the new Hugging Face Deep Learning Container. 
    Jeff: ok.
    Jeff: and how can I get started? 
    Jeff: where can I find documentation? 
    Philipp: ok, ok you can find everything here. https://huggingface.co/blog/the-partnership-amazon-sagemaker-and-hugging-face                                           
    '''

data= {"inputs":conversation}

predictor.predict(data)